# Fase 3 - *Intervals*
## Definição dos intervalos de corte para cada arquivo

In [6]:
import os
import ipywidgets as widgets
from src.setup.Paths import Paths
from src.data.trunc_intervals import TruncIntervals
from src.data.signal_handler import Plotter

### Definição dos caminhos dos recursos e dos canais de dados utilizados

In [7]:
files_path = Paths.get_resource_path(Paths.Resources.FORMATTED)
trunc_intervals_path = Paths.get_resource_path(Paths.Resources.TRUNC_JSONS)
channels = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

### Recuperando arquivos de sinal em `FORMATTED`

In [8]:
csv_filenames = [file for file in os.listdir(files_path) if '.csv' in file]

if len(csv_filenames) == 0:
    print('Sem dados no diretório especificado.')
    exit()

### Criação dos widgets

#### Criação de flags de estado dos widgets de arquivo e canal

In [9]:
file_widget_changed = False
channel_widget_changed = False

#### Criação do widget de output

In [10]:
output = widgets.Output()

#### Criação dos widgets de seleção de arquivo, canal e botão de refresh

In [11]:
file_w = widgets.Combobox(options = csv_filenames, value = csv_filenames[0], description = 'Arquivo', layout = widgets.Layout(width = '300px'))
channel_w = widgets.Dropdown(options = channels, value = channels[0], description = 'Canal', layout = widgets.Layout(width = '300px'))
refresh_btn_w = widgets.Button(description = 'Refresh', layout = widgets.Layout(width = '70px'))
refresh_btn_w.layout.margin = '0px 0px 0px 50px'

signal_controller_hboxw = widgets.HBox([file_w, channel_w, refresh_btn_w])

#### Criação dos widgets para lidar com os intervalos de corte

In [12]:
label_w = widgets.Label(value = 'Intervalo de corte')
bottom_limit_w = widgets.FloatText(layout = widgets.Layout(width = '100px'))
top_limit_w = widgets.FloatText(layout = widgets.Layout(width = '100px'))
add_interval_btn_w = widgets.Button(description = 'Add', layout = widgets.Layout(width = '70px'))
pop_interval_btn_w = widgets.Button(description = 'Pop', layout = widgets.Layout(width = '70px'))
save_interval_btn_w = widgets.Button(description = 'Save', layout = widgets.Layout(width = '70px'))

trunc_intervals_hboxw = widgets.HBox([label_w, bottom_limit_w, top_limit_w, add_interval_btn_w, 
                                         pop_interval_btn_w, save_interval_btn_w], layout=widgets.Layout(justify_content='center'))

### Instanciando as classes `TruncIntervals` e `Plotter`

In [13]:
trunc_intervals = TruncIntervals(trunc_intervals_path)
plotter = Plotter(files_path, output)

### Definição de eventos

#### Handlers para monitoramento dos widgets de arquivo e canal

In [14]:
def file_widget_change_handler(change):
    global file_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        file_widget_changed = True

In [15]:
def channel_widget_change_handler(change):
    global channel_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        channel_widget_changed = True 

#### Eventos de click

In [16]:
def refresh_button_clicked(btn):
    global file_widget_changed
    global channel_widget_changed
    if file_widget_changed:
        trunc_intervals.save_current_file_intervals()
        trunc_intervals.load_file_intervals(file_w.value)
        plotter.load_signal(file_w.value, channel_w.value)
        plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))
        file_widget_changed = False
        channel_widget_changed = False
    elif channel_widget_changed:
        plotter.change_current_fig(channel_w.value)
        plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))
        channel_widget_changed = False

In [17]:
def add_intervals_clicked(btn):
    start = bottom_limit_w.value
    end = top_limit_w.value
    trunc_intervals.add_interval_by_channel(channel_w.value, start, end)
    plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

In [18]:
def pop_intervals_clicked(btn):
    trunc_intervals.pop_interval_by_channel(channel_w.value)
    plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

In [19]:
def save_intervals_clicked(btn):
    trunc_intervals.save_current_file_intervals()

### Associação dos widgets aos seus eventos

In [20]:
file_w.observe(file_widget_change_handler)
channel_w.observe(channel_widget_change_handler)

refresh_btn_w.on_click(refresh_button_clicked)
add_interval_btn_w.on_click(add_intervals_clicked)
pop_interval_btn_w.on_click(pop_intervals_clicked)
save_interval_btn_w.on_click(save_intervals_clicked)

### Display do editor de intervalos

In [21]:
trunc_intervals.load_file_intervals(file_w.value)
plotter.load_signal(file_w.value, channel_w.value)
plotter.plot_signal(trunc_intervals.get_channel_intervals(channel_w.value))

display(signal_controller_hboxw, output, trunc_intervals_hboxw)

Output()